In [1]:
from random import randint
from collections import defaultdict
from uuid import uuid4
from dataclasses import dataclass
import json

import matplotlib.pyplot as plt
from ipynb.fs.full.Observer import RemoteCamera, CaptureMachine, TrackedObject, CaptureConfiguration, \
    Camera, hStackImages, vStackImages, cameras
import dma.MechaCombat as mc
QuantumSystem = mc.QuantumSystem

In [2]:
@dataclass
class QuantumObject(TrackedObject):
    name: str
    objectType: str

    def __eq__(self, other):
        return super().__eq__(other)
    
    def __post_init__(self):
        super().__post_init__()
        self.qid = str(uuid4())

    def previousVersion(self):
        return type(self)({camNum: change.lastChange if change is not None else None for camNum, change in self.changeSet.items()}, self.name, self.objectType)

    def __repr__(self):
        changeSet = {camNum: cS for camNum, cS in self.changeSet.items() if cS is not None and cS.changeType not in [None, 'delete']}
        return f"QO - {self.objectType} - {self.name} ({changeSet})"

In [6]:
class HarmonyMachine(CaptureMachine):
    states = ["idle", "unstable", "classify", "illegal"]
    modes = ["passive", "calibrate", "add", "move", "action"]
    MM_FOR_MOVE = 15

    def __init__(self, captureConfiguration: CaptureConfiguration):
        super().__init__(captureConfiguration)
        self.hc = self.cc
        mc.qs.reset()
        self.newObjectBuffer = []
        self.selectedObjects = []
    
    def isLegal(self, classifiedChange):
        return True
        
    def storeNewObject(self, objDef):
        self.lastMemory = objDef
        if not objDef.isNewObject:
            existingIndex = self.newObjectBuffer.index(objDef.previousVersion())
            existingObj = self.newObjectBuffer[existingIndex]
            existingObj.update(objDef, overwrite=self.cc.rsc.trackedObjectLastDistance(objDef) > self.MM_FOR_MOVE)
        else:
            self.newObjectBuffer.append(objDef)
        
        for camNum, change in objDef.changeSet.items():
            cameras[camNum].commitChange(change)
        
        self.transitions.append({
            "obj": objDef,
            "cycle": self.cycleCounter,
            "cameraChanges":  {camNum: {"ref": cam.referenceFrame, "fin": cam.mostRecentFrame}
                               for camNum, cam in cameras.items()}})
    
    def commitChanges(self, objDef):
        try:
            existingIndex = self.memory.index(objDef.previousVersion())
            existingObj = self.memory[existingIndex]
            print(f"Updating Memory {existingIndex}")
            existingObj.update(objDef, overwrite=self.cc.rsc.trackedObjectLastDistance(objDef) > self.MM_FOR_MOVE)
        except Exception:
            raise Exception("Cannot commit to unrecognized memory")
        self.lastMemory = existingObj
        
        for camNum, change in objDef.changeSet.items():
            cameras[camNum].commitChange(change)

        self.transitions.append({
            "obj": objDef,
            "cycle": self.cycleCounter,
            "cameraChanges":  {camNum: {"ref": cam.referenceFrame, "fin": cam.mostRecentFrame}
                               for camNum, cam in cameras.items()}})

    def annotateObject(self, oid, objName, objType):
        caps = {cap.oid: cap for cap in self.newObjectBuffer}
        cap = caps[oid]
        center = self.cc.rsc.changeSetToRealCenter(cap)
        qObj = QuantumObject(cap.changeSet, objName, objType)
        factory = mc.ObjectFactories[objType]
        factory(objName)
        mc.XYLocation.set_location(objName, json.dumps(list(center) + [0]))
        self.memory.append(qObj)
        self.newObjectBuffer.remove(cap)
    
    def cycle(self):
        try:
            print(f"Starting Cycle {self.cycleCounter:5} -- {self}")
            self.cycleCounter += 1
            nextState = "idle"
            self.hc.capture()
            if self.mode == "passive":
                self.hc.setReference()
                return
            # else: mode in ["calibrate", "add", "move", "attack"]

            changes = self.referenceFrameDeltas()
            classification = None

            if changes.empty and self.state == "idle":
                self.hc.setReference()
            elif not changes.empty:
                nextState = "unstable"
                if self.state in ["unstable", "classify"] and changes == self.lastChanges:  # TODO: and cameraChangeOverlap
                    classification = self.classifyChanges(changes)
                    nextState = "classify"
                    if self.state == "classify":
                        if self.mode == "action":
                            assert not classification.isNewObject, f"Cannot add objects in {self.mode}"
                        elif self.mode in ["move", "action"]:
                            assert not classification.isNewObject, f"Cannot add objects in {self.mode}"
                            if self.mode == "move":
                                assert self.isLegal(classification), f"{classificaton} is an illegal move"
                            elif self.mode == "action":
                                assert self.cc.rsc.trackedObjectLastDistance(classification) < self.MM_FOR_MOVE, f"{classification} moves too far for selection"
                                
                            nextState = "idle"
                            self.commitChanges(classification)
                            if self.mode == "action":
                                self.selectedObjects = list(set(self.selectedObjects.append(self.lastMemory)))
                                if len(self.selectedObjects) == 2:
                                    pass
                                    # Test Legality of Attack between sO[0] -> sO[1]
                                    # if legal, Calculate SATOR for Attack
                            self.hc.setReference()
                        elif self.mode == "add":
                            nextState = "idle"
                            self.storeNewObject(classification)
                            self.hc.setReference()
                        elif self.mode == "calibrate":
                            try:
                                self.calibrateToObject(classification)
                            except AssertionError as ae:
                                print(f"Failed Calibration: {ae}")
                            nextState = "idle"
                            self.hc.setReference()
                        else:
                            raise Exception("Unrecognized State")
            self.state = nextState
            self.lastChanges = changes
            self.lastClassification = classification
        except:
            from traceback import format_exc
            print("CYCLE FAILURE!!!")
            print(format_exc())
            self.state = "unstable"
            self.lastChanges = None
            self.lastClassification = None
    
    def actionMode(self):
        self.selectedObjects = []
        self.mode = "action"
    
    def moveMode(self):
        self.mode = "move"
    
    def addMode(self):
        self.mode = "add"
    
    def passiveMode(self):
        self.mode = "passive"
    
    def startCalibration(self):
        self.calibrationPts = []
        self.mode = "calibrate"
    
    def abortCalibration(self):
        self.calibrationPts = []
        self.mode = "passive"
            
    def cycleForAddition(self):
        self.mode = "add"
        startLen = len(self.newObjectBuffer)
        while len(self.newObjectBuffer) == startLen:
            self.cycle()
        self.mode = "idle"
        return self.newObjectBuffer[-1]
            
    def cycleForChange(self):
        self.mode = "move"
        startLen = len(self.transitions)
        while len(self.transitions) == startLen:
            self.cycle()
        self.mode = "idle"
        return self.lastMemory
            
    def cycleForAction(self):
        self.mode = "action"
        startLen = len(self.transitions)
        while len(self.transitions) < startLen + 2:
            self.cycle()
        self.mode = "idle"
        return self.lastMemory
            
    def __repr__(self):
        return f"CapMac -- {self.mode} {str(len(self.calibrationPts)) + ' ' if self.mode == 'calibrate' else ''}{self.state}"

In [7]:
cc = CaptureConfiguration()
cm = HarmonyMachine(cc)

In [ ]:
if __name__ == "__main__":
    for i in range(3):
        cm.cc.capture()
    print("Waiting for object...")
    obj = cm.cycleForAddition()
    cm.annotateObject(obj.oid, "Foo", "Huntsman")
    plt.imshow(obj.visual())

In [ ]:
if __name__ == "__main__":
    for i in range(2):
        print("Waiting for movement")
        obj = cm.cycleForChange()
        plt.imshow(obj.visual())
        plt.show()